# VacationPy

In [1]:
####################################################
#00     I/O                                        #
#   a- import libraries and api key.               #
#   b- read input into df and configure api key.   #
####################################################

#a
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from api_keys import g_key

#b
city_weather_file = "output_data/cities.csv"
city_weather = pd.read_csv(city_weather_file)

gmaps.configure(api_key=g_key)

In [2]:
#############################################################
#01     Humidity Map                                        #
#   a- set locations and humidity as "heat" for map.        #
#   b- figure base map, set heat layer, and add it.         #
#   c- display map and save.                                #
#############################################################

#a
locations = city_weather[["lat", "lng"]].astype(float)
humidity = city_weather["humid"]#.astype(float)

#b
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.7)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=100, point_radius = 2)
fig.add_layer(heat_layer)

#c
fig

Figure(layout=FigureLayout(height='420px'))

In [3]:
################################################################
#02      Vacation Weather                                      #
#   a- create new df, filtering cities to my weather           #
#      specifications for vacation spots.                      #
#   b- drop any rows with missing data from new vacation df.   #
################################################################

#a
vacation_weather = city_weather[(city_weather["temp"] > 80) & 
                                (city_weather["clouds"] >= 50) & (city_weather["humid"] >= 85)]
#b
vacation_weather = vacation_weather.dropna()
vacation_weather.reset_index(inplace=True,drop=True)
vacation_weather

,city,temp,humid,wind,clouds,lat,lng,maxtemp,country,local_date_time
0,san patricio,80.65,89,8.05,90,28.0170,-97.5169,82.4,US,2021-05-10 17:12:41
1,sembakung,80.60,94,2.30,75,3.8167,117.2500,80.6,ID,2021-05-11 06:17:15
2,nuevo progreso,81.00,91,1.01,100,23.1667,-97.9500,81.0,MX,2021-05-10 17:17:23


In [4]:
############################################################################
#03    Hotel Df                                                            #
#   a- creaate hotel_df, add name col, set base_url and parameters.        #
#      create list in case missing hotel info to delete row.               #
#   b- for each location in df, call google for nearby hotel, add to df.   #
#   c- if missing hotel info, delete records from hotels_df.               #
############################################################################

#a
hotel_df = vacation_weather
hotel_df["Hotel Name"] = ""
params = {"radius": 5000, "keyword": "hotels", "key": g_key}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
del_indexes = []

#b
for x, row in hotel_df.iterrows():
    lat = row["lat"]                                           # get lat, lng from df
    lng = row["lng"]
    params["location"] = f"{lat},{lng}"                        # set location at each iteration 
    
    response = requests.get(base_url, params=params).json()    #call google for hotel name
    
    try:
        hotel_df.loc[x, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        del_indexes.append(x)
#c
if len(del_indexes) > 0:
    hotel_df.drop(del_indexes, inplace=True)
    hotel_df.reset_index(inplace=True, drop=True)

In [5]:
##############################################################
#04     Hotel InfoBox                                        #
#   - prewritten block of code for step 05 Marker Overlay.   #
#      - setup infobox data in hotel_info.                   #
#      - create lat,lng df for marker locations.             #
##############################################################

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [6]:
###################################################
#05      Marker Overlay                           #
#   a- assign marker layer to variable markers.   #
#   b- add layer to map and display.              #
###################################################

#a
markers = gmaps.marker_layer(locations,info_box_content =hotel_info)
#b
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))